In [ ]:
###### Set Up #####
# verify our folder with the data and module assets is installed
# if it is installed make sure it is the latest
!test -e ds-assets && cd ds-assets && git pull && cd ..
# if it is not installed clone it 
!test ! -e ds-assets && git clone https://github.com/IndraniMandal/ds-assets.git
# point to the folder with the assets
home = "ds-assets/assets/" 
import sys
sys.path.append(home) 

Cloning into 'ds-assets'...
remote: Enumerating objects: 171, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 171 (delta 1), reused 2 (delta 0), pack-reused 164
Receiving objects: 100% (171/171), 7.40 MiB | 12.84 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [ ]:
# preamble to be able to run notebooks in Jupyter and Colab
try:
    from google.colab import drive
    import sys
    
    drive.mount('/content/drive')
    notes_home = "/content/drive/My Drive/CSC310/notes/"
    user_home = "/content/drive/My Drive/"
    
    sys.path.insert(1,notes_home) # let the notebook access the notes folder

except ModuleNotFoundError:
    notes_home = "" # running native Jupyter environment -- notes home is the same as the notebook
    user_home = ""  # under Jupyter we assume the user directory is the same as the notebook

Mounted at /content/drive


#Evualating Models

###Libaries or modules import

In [ ]:
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
# sklearn provides manipulation of training sets
# here we do train/test split
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
from treeviz import tree_print
import numpy as np
from matplotlib import pyplot as plt
from google.colab import files
data_to_load = files.upload()
import io

Saving Backpack.csv to Backpack.csv


In [ ]:
file_name = "Backpack.csv"
backpack_df = pd.read_csv(io.BytesIO(data_to_load[file_name]))

In [ ]:
backpack_df.head()

,Unnamed: 0,BackpackWeight,BodyWeight,Ratio,BackProblems,Major,Year,Sex,Status,Units
0,1,9,125,0.072000,1,Bio,3,Female,U,13
1,2,8,195,0.041026,0,Philosophy,5,Male,U,12
2,3,10,120,0.083333,1,GRC,4,Female,U,14
3,4,6,155,0.038710,0,CSC,6,Male,G,0
4,5,8,180,0.044444,0,EE,2,Female,U,14


#Cleanning the data frame

##Printing out all the columns founded within backpack_df

In [ ]:
backpack_df.columns

Index(['Unnamed: 0', 'BackpackWeight', 'BodyWeight', 'Ratio', 'BackProblems',
       'Major', 'Year', 'Sex', 'Status', 'Units'],
      dtype='object')

##Viewing a how the backpack data frame looks 

In [ ]:
updated_backpack_df = backpack_df.drop(["Unnamed: 0", "Status", "Major"], axis = 1)
# back_pack = backpack_df.Sex.replace(["Female", "Male"], [0,1])
updated_backpack_df.head()

,BackpackWeight,BodyWeight,Ratio,BackProblems,Year,Sex,Units
0,9,125,0.072000,1,3,Female,13
1,8,195,0.041026,0,5,Male,12
2,10,120,0.083333,1,4,Female,14
3,6,155,0.038710,0,6,Male,0
4,8,180,0.044444,0,2,Female,14


Changing all the sex catgories with zero and one to make this column numerical instead of categorical

In [ ]:
updated_backpack_df.Sex = updated_backpack_df.Sex.replace(["Female", "Male"], [0,1])

In [ ]:
updated_backpack_df.head()

,BackpackWeight,BodyWeight,Ratio,BackProblems,Year,Sex,Units
0,9,125,0.072000,1,3,0,13
1,8,195,0.041026,0,5,1,12
2,10,120,0.083333,1,4,0,14
3,6,155,0.038710,0,6,1,0
4,8,180,0.044444,0,2,0,14


In [ ]:
np.set_printoptions(formatter={'float_kind':"{:3.2f}".format})
import numpy as np
from sklearn.model_selection import cross_val_score

X = updated_backpack_df.drop(["BackProblems"], axis = 1)
y = updated_backpack_df["BackProblems"]

# set up the model
model = tree.DecisionTreeClassifier(criterion='entropy')

# do the 5-fold cross validation
scores = cross_val_score(model, X, y, cv=5)
print("Fold Accuracies: {}".format(scores))

Fold Accuracies: [0.40 0.50 0.60 0.60 0.55]


printing out all the columns and rows within the updated_backpack_df

In [ ]:
updated_backpack_df.shape

(100, 7)

In [ ]:
# compute 95% confidence intervals for classification and regression
# problems

def classification_confint(acc, n):
    '''
    Compute the 95% confidence interval for a classification problem.
      acc -- classification accuracy
      n   -- number of observations used to compute the accuracy
    Returns a tuple (lb,ub)
    '''
    import math
    interval = 1.96*math.sqrt(acc*(1-acc)/n)
    lb = max(0, acc - interval)
    ub = min(1.0, acc + interval)
    return (lb,ub)

def regression_confint(rs_score, n, k):
    '''
    Compute the 95% confidence interval for a regression problem.
      rs_score -- R^2 score
      n        -- number of observations used to compute the R^2 score
      k        -- number of independent variables in dataset
    Returns a tuple (lb,ub)
    Reference:
    https://books.google.com/books?id=gkalyqTMXNEC&pg=PA88#v=onepage&q&f=false
    '''
    import math
    interval = 2*math.sqrt((4*rs_score*(1-rs_score)**2*(n-k-1)**2)/((n**2 - 1)*(n+3)))
    lb = max(0, rs_score - interval)
    ub = min(1.0, rs_score + interval)
    return (lb,ub)

In [ ]:
observations = 100
acc = .60
lb,ub = classification_confint(acc,observations)
print('Accuracy: {} ({:3.2f},{:3.2f})'.format(acc,lb, ub))

Accuracy: 0.6 (0.50,0.70)


In [ ]:
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split



X = updated_backpack_df.drop(["BackProblems"], axis = 1)
y = updated_backpack_df["BackProblems"]
# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=2)

# set up the tree model object - limit the complexity to put us somewhere in the middle of the graph.
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=1)

# fit the model on the training set of data
model.fit(X_train, y_train)

# Test results: evaluate the model on the testing set of data
y_test_model = model.predict(X_test)
acc = accuracy_score(y_test, y_test_model)
observations = X_test.shape[0]
lb,up = classification_confint(acc, observations)
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

Accuracy: 0.65 (0.44,0.70)


##Although this model doesn't has significant high accuracy, this is the best accuracy as the more I increased the max_depth the model became less accurate

In [ ]:
rs_score = .65
observations = 100
variables = 6 # independent variables

lb,ub = regression_confint(rs_score, observations, variables)
print("R^2 Score: {:3.2f} ({:3.2f}, {:3.2f})".format(rs_score,lb,ub))

R^2 Score: 0.65 (0.55, 0.75)


In [ ]:
import numpy as np
import pandas
from sklearn.tree import DecisionTreeRegressor
#from assets.confint import regression_confint

# get our dataset

# build model object
model = DecisionTreeRegressor(max_depth=None)

# fit model
# We have to reshape the values array to make 'fit' happy because
# the array only has a single feature
#BackpackWeight
model.fit(updated_backpack_df["BackProblems"].values.reshape(-1,1),updated_backpack_df)

# R^2 score
rs_score = model.score(updated_backpack_df["BackProblems"].values.reshape(-1,1),updated_backpack_df)
observations = updated_backpack_df.shape[0]
variables = 1
lb,ub = regression_confint(rs_score, observations, variables)

# print out R^2 score with its 95% confidence interval
print("R^2 Score: {:3.2f} ({:3.2f}, {:3.2f})".format(rs_score,lb,ub))

R^2 Score: 0.17 (0.03, 0.30)


#Statistical Significance

In [ ]:
#from assets.confint import classification_confint

observations = 100

# first classifier
acc1 = .40
lb1,ub1 = classification_confint(acc1,observations)
print('Accuracy: {} ({:3.2f},{:3.2f})'.format(acc1,lb1, ub1))

# second classifier
acc2 = .60
lb2,ub2 = classification_confint(acc2,observations)
print('Accuracy: {} ({:3.2f},{:3.2f})'.format(acc2,lb2, ub2))

Accuracy: 0.4 (0.30,0.50)
Accuracy: 0.6 (0.50,0.70)
